In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LinearRegression
from keras.layers import *
from keras.models import Sequential
import numpy as np

Using TensorFlow backend.


### Load Dataset

In [2]:
### Labels
labels = {
   'New York Times': -0.45,
   'Breitbart': 1,
   'CNN': -0.4,
   'Business Insider': -0.3,
   'Atlantic': -0.6,
   'Fox News': 0.8,
   'Talking Points Memo': -0.6,
   'Buzzfeed News': -0.7,
   'National Review': 0.8,
   'New York Post': 0.5,
   'Guardian': -0.5,
   'NPR': -0.5,
   'Reuters': 0,
   'Vox': -0.85,
   'Washington Post': -0.7,
}

In [3]:
os.listdir('local_cache')

['articles1.csv', 'articles2.csv', 'articles3.csv']

In [4]:
df = pd.concat([pd.read_csv('local_cache/'+str(x), index_col=0) for x in os.listdir('local_cache')])

In [5]:
df = df[['title', 'publication', 'author', 'content']]

In [6]:
df['labels'] = df.apply(lambda x: labels[x['publication']], axis=1)

In [7]:
df = df.sample(frac=.5)

In [8]:
# 60% Training
# 20% Test
# 20% Validation
msk = np.random.rand(len(df)) < 0.6
train = df[msk]
tmp = df[~msk]
msk = np.random.rand(len(tmp)) < 0.5
test = tmp[msk]
val = tmp[~msk]

## Bag of Words Features

In [16]:
count_vect = CountVectorizer(stop_words='english',
                             ngram_range=(1, 5),
                             strip_accents='ascii',
                             max_features=40000)
X_train_counts = count_vect.fit_transform(tqdm(train['content']))


 16%|█▌        | 6658/42572 [00:15<01:25, 421.49it/s]


 31%|███▏      | 13349/42572 [00:31<01:09, 422.91it/s]


 46%|████▋     | 19691/42572 [00:47<00:55, 411.71it/s]


 61%|██████▏   | 26152/42572 [01:03<00:39, 414.47it/s]


 77%|███████▋  | 32575/42572 [01:18<00:24, 416.14it/s]


 92%|█████████▏| 39310/42572 [01:35<00:07, 412.52it/s]


100%|██████████| 42572/42572 [01:42<00:00, 414.49it/s]

In [17]:
tf_transformer = TfidfTransformer().fit(X_train_counts)

In [18]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_val_counts = count_vect.transform(tqdm(val['content']))
X_val_tfidf = tfidf_transformer.transform(X_val_counts)


 49%|████▉     | 7027/14225 [00:15<00:15, 453.92it/s]


 99%|█████████▉| 14079/14225 [00:30<00:00, 456.25it/s]


100%|██████████| 14225/14225 [00:31<00:00, 456.08it/s]

In [19]:
model = Sequential()
model.add(Dense(2455, input_shape=(X_train_tfidf.shape[1], )))
model.add(Dense(1000))
model.add(Dense(100))
model.add(Dense(1))
model.compile('SGD', 'MAE')

In [20]:
train.head()

,title,publication,author,content,labels
23056,"Brazil Nears 100,000 Zika Cases Less Than 100 ...",Breitbart,Frances Martel,Brazil confirms this week it has documented a ...,1.0
20419,Breitbart News CEO &amp President Responds to ...,Breitbart,John Nolte,Michelle Fields is scheduled to be on “Good Mo...,1.0
73666,Donald Trump Lies That He Opposed Iraq War Fr...,Buzzfeed News,Kyle Blaine,During NBC News’ presidential forum on Wednes...,-0.7
82804,Judge’s blocking of travel ban sets stage for ...,New York Post,Joe Tacopino and Daniel Halper,A federal judge in Seattle Friday struck down ...,0.5
114928,"Americans Don’t Like Caucuses, But Replacing T...",NPR,Megan Verlee,"This week, as part of the Nation Engaged proje...",-0.5


In [21]:
model.fit(X_train_tfidf, np.array(train['labels'].tolist()).reshape(-1, 1),
          validation_data=())

Epoch 1/1
42572/42572 [==============================] - 381s 9ms/step - loss: 0.5226


In [ ]:
clf = LinearRegression().fit(X_train_tfidf, np.array(train['labels'].tolist()).reshape(-1, 1))

## Test data

In [ ]:
docs_new = test['content']
X_new_counts = count_vect.transform(tqdm(docs_new))
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [ ]:
predicted = clf.predict(X_new_tfidf)

In [ ]:
np.array(test['labels'].tolist()).reshape(-1, 1)

In [ ]:
(predicted-np.array(test['labels'].tolist()).reshape(-1, 1)).mean()